In [1]:
# 引入套件
import pandas as pd
import numpy as np

# 讀取 json 檔案包含轉換向量
df = pd.read_json("../backup/vector_all.json")

In [ ]:
def search_keyword(keyword):
    # Replace NaN values with empty strings
    df['TF'] = df['TF'].fillna('')
    # Search for the keyword in the TF column
    results = df[df['TF'].str.contains(keyword)]['TF'].tolist()

    # Split the results into individual words and count their occurrences
    word_counts = {}
    for result in results:
        words = result.split()
        for word in words:
            word_counts[word] = word_counts.get(word, 0) + 1

    # Sort the words based on their occurrence count in descending order
    sorted_words = sorted(word_counts.keys(),
                          key=lambda x: word_counts[x], reverse=True)

    # Return the top 3 words
    if (len(sorted_words) > 3):
        sorted_words = sorted_words[:3]
    return sorted_words

In [ ]:
def create_result_table(words_list):
    # 定義一個空的表格
    table_data = []
    # 定義一個空的錯誤列表
    errList = []
    # 逐一取出文字列表中的文字
    for word in words_list:
        try:
            # 將文字進行 search_keyword(文字) 並將結果存入 result
            result = search_keyword(word)
            # 將 word 和 result 存入表格中
            table_data.append({
                'word': word,
                'simliar_word': result,
            })
        except:
            # 如果有錯誤，將 word 存入錯誤列表中
            errList.append(word)
            continue

    # 將表格和錯誤列表回傳
    return {
        "data": table_data,
        "errList": errList,
    }


# 从 TF 列获取文字列表
words_list = df['TF'].str.split().explode().unique().tolist()

# 利用文字列表中的文字進行 search_keyword(文字) 并将结果输出成表格
result_table = create_result_table(words_list)
simliar_word_df = pd.DataFrame(result_table["data"]).fillna("")
# 將 simliar_word 列中的列表轉成字串
simliar_word_df["simliar_word_str"] = simliar_word_df["simliar_word"].apply(
    lambda x: " ".join(x))

In [ ]:
simliar_word_df

In [ ]:
from pymongo import MongoClient

# 建立 MongoDB 連線
client = MongoClient("mongodb://100.90.12.119:27017/")
db = client.nthu_trello_helper
mongo_collection = db.close_words

In [ ]:
for idx, row in simliar_word_df.iterrows():
    # 插入 MongoDB
    mongo_collection.insert_one({
        "key": str(row['word']),
        "value": row['simliar_word_str']
    })

In [ ]:
mongo_collection.insert_one({
    "key": "虎克定律",
    "value": "虎克定律 Hooke’s law 應力 應變 運動與力"
})

In [ ]:
input_keyword = "藍光"
output = search_keyword(input_keyword)
print(output)